In [141]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
import chart_studio.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as offline
import json

In [142]:
#import median family income files 2017-2021
fy2017 = pd.read_excel('~/Downloads/S235-S236-FY17.xlsx')
fy2018 = pd.read_excel('~/Downloads/S235-S236-FY18.xlsx')
fy2019 = pd.read_excel('~/Downloads/S235-S236-FY19.xlsx')
fy2020 = pd.read_excel('~/Downloads/S235-S236-FY20.xlsx')
fy2021 = pd.read_excel('~/Downloads/S235-S236-FY21.xlsx')

In [143]:
#drop data for cities that are not metro areas
fy2017 = fy2017[['Metro_Area_Name','median2017']].loc[fy2017['metro'] == 1]
fy2018 = fy2018[['Metro_Area_Name','median2018']].loc[fy2018['metro'] == 1]
fy2019 = fy2019[['Metro_Area_Name','median2019']].loc[fy2019['metro'] == 1]
fy2020 = fy2020[['Metro_Area_Name','median2020']].loc[fy2020['metro'] == 1]
fy2021 = fy2021[['Metro_Area_Name','median2021']].loc[fy2021['metro'] == 1]

In [144]:
#drop duplicate data
fy2017 = fy2017.drop_duplicates()
fy2018 = fy2018.drop_duplicates()
fy2019 = fy2019.drop_duplicates()
fy2020 = fy2020.drop_duplicates()
fy2021 = fy2021.drop_duplicates()

In [145]:
#merge median family income files on metro name
all_data = fy2017.merge(fy2018, on='Metro_Area_Name')

In [146]:
all_data = all_data.merge(fy2019, on='Metro_Area_Name')

In [147]:
all_data = all_data.merge(fy2020, on='Metro_Area_Name')

In [148]:
all_data = all_data.merge(fy2021, on='Metro_Area_Name')

In [149]:
all_data.shape

(622, 6)

In [150]:
all_data.head()

,Metro_Area_Name,median2017,median2018,median2019,median2020,median2021
0,"Montgomery, AL MSA",59700,62900,65900,65700,67600
1,"Daphne-Fairhope-Foley, AL MSA",61500,69400,80500,81000,74300
2,"Birmingham-Hoover, AL HUD Metro FMR Area",63100,71000,74400,73100,78000
3,"Anniston-Oxford-Jacksonville, AL MSA",53000,56800,56200,61500,63700
4,"Chilton County, AL HUD Metro FMR Area",52700,54600,55300,56100,56700


In [151]:
#export cleaned family incomes data
all_data.to_csv('familyincomes.csv')

In [121]:
#import metro data 
#(this is a csv from data combined in a google spreadsheet using the family income data from the above csv and zillow)
metro_data = pd.read_csv('~/Downloads/metrodatas.csv')

In [122]:
#add id to data -- this doesn't end up getting used
metro_data['id'] = metro_data.index

In [123]:
#convert data from wide to long (one col for years) to get min/max later
metro_data_long = pd.wide_to_long(metro_data, stubnames = ["PIratio","Home"], i='ZillowName', j='year').reset_index()

In [124]:
metro_data_long.head()

,ZillowName,year,Longitude,id,Home Change 2000 to 21,Latitude,Home Change 2008 to 2021,Metropolitan statistical area,PIratio,Home
0,"Los Angeles-Long Beach-Anaheim, CA",2000,-118.182750,0,1.016030,34.108703,0.267929,"Los Angeles-Long Beach-Anaheim, CA MSA",4.852012,401467.98
1,"San Jose, CA",2000,-121.371370,1,0.488040,36.908472,0.328905,"San Jose-Sunnyvale-Santa Clara, CA MSA",6.109914,844200.58
2,"San Francisco, CA",2000,-122.274430,2,0.469275,37.773718,0.237575,"San Francisco-Oakland-Berkeley, CA MSA",5.647744,671812.54
3,"San Diego, CA",2000,-116.776120,3,0.598213,33.023604,0.367383,"San Diego-Chula Vista-Carlsbad, CA MSA",4.853127,413892.20
4,"New York, NY",2000,-73.873758,4,0.488695,40.774930,-0.098756,"New York-Newark-Jersey City, NY-NJ-PA MSA",4.426770,395106.91


In [125]:
#get data for price-to-income map
metro_data_df = metro_data[['Metropolitan statistical area','id','ZillowName','Latitude','Longitude','PIratio2000', 'PIratio2008','PIratio2017','PIratio2018','PIratio2019','PIratio2020','PIratio2021']]

In [126]:
#check for NaN values
metro_data_df.isna().any()

Metropolitan statistical area    False
id                               False
ZillowName                       False
Latitude                         False
Longitude                        False
PIratio2000                       True
PIratio2008                      False
PIratio2017                      False
PIratio2018                      False
PIratio2019                      False
PIratio2020                      False
PIratio2021                      False
dtype: bool

In [127]:
#only NaN value was for Indianapolis in 2000, so fillig it with the 2000 PI median
metro_data_df['PIratio2000'].fillna(metro_data_df['PIratio2000'].median(),inplace=True)

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [128]:
metro_data_df.isna().any()

Metropolitan statistical area    False
id                               False
ZillowName                       False
Latitude                         False
Longitude                        False
PIratio2000                      False
PIratio2008                      False
PIratio2017                      False
PIratio2018                      False
PIratio2019                      False
PIratio2020                      False
PIratio2021                      False
dtype: bool

In [134]:
#create map using plotly with a slider that updates the map each year in the dataset

fig = go.Figure()

col_name = metro_data_df.columns
n_col = len(metro_data_df.columns)
date_list =[]
col_ratio = 5 #start column for slider loop (i.e., starts price-to-income data)
n_range = 7 #total number of columns

#loop through columns to create data for each year to add to map
for i in range(n_range):
    df = metro_data_df[['Latitude','Longitude', 'ZillowName', col_name[col_ratio]]]
    
    lat = df['Latitude']
    lon = df['Longitude']
    

    df['text'] = df.apply(lambda x: x[2] + '<br> Price to income: ' + str(round(x[3],1)), axis=1) #pop up text on metro

    #add trace using individual lat/long for each metro in each year 
    fig.add_trace(
    go.Scattergeo(
        name='',
        lon=lon,
        lat=lat,
        visible=False,
        hovertemplate=df['text'],
        showlegend=False,
        mode='markers',
        marker=dict(
            size=15, #metro icon size
            opacity=0.6, #metro icon opacity
            symbol='circle', #metro icon style
            reversescale = True,
            autocolorscale = False,
            colorscale = 'RdBu', #color scale for color bar
            cmin = metro_data_long['PIratio'].min(), #sets min for this year as min across all years
            color = df[col_name[col_ratio]], #sets color to the individual PI ratio for that metro
            cmax = metro_data_long['PIratio'].max(), #sets max for this year as max across all years
            colorbar_title = "Price to income ratio", #title of the color bar legend
            colorbar_dtick=.5, #step increment for color bar legend
            ),
        )
    )
    
    col_ratio += 1
    
steps =[]
years = metro_data_long['year'].sort_values().unique() #years available in the dataset in chron order as list

for i in range(len(fig.data)):
    pi_avg = metro_data_long['PIratio'].loc[metro_data_long['year'] == years[i]].mean() #get average ratio for the year
    step = dict(
        method='update', #update step lets you update data and layout
        args = [
            {'visible': [False] * len(fig.data)}, #set all data to false visibility
            {'title': "<b>Price to Income Ratio by Metro</b> <br><i>{0} Average: {1}</i>".format(years[i], round(pi_avg,2))}, #update title to include average for that year
        ],
        label = 'Year {}'.format(years[i]),
    )
    
    step["args"][0]["visible"][i] = True #set visibility of true for appropriate column for specific year
    steps.append(step)
    
#create slider using steps
sliders = [dict(
    active=0,
    currentvalue = {'prefix':'Date: '},
    pad = {'t':1},
    steps = steps,
)]


fig.data[0].visible = True
min_year = metro_data_long['year'].min()
pi_avg = metro_data_long['PIratio'].loc[metro_data_long['year'] == min_year].mean()

fig.update_layout(
    title = "<b>Price to Income Ratio by Metro</b> <br><i>{0} Average: {1}</i>".format(min_year, round(pi_avg,2)), #set initial title for first year in data
    sliders=sliders,
    geo = dict(
        scope = 'usa',
        projection_type = 'albers usa',
        showland = True,
    )
)

fig.show()
#offline.plot(fig, filename='testratio2.html') #download as html file

/Users/francescaortegren/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [135]:
#get home value data for map 
df2 = metro_data[['Metropolitan statistical area', 'id', 'ZillowName', 'Latitude', 'Longitude','Home2000', 'Home2008','Home2017', 'Home2018', 'Home2019', 'Home2020',
       'Home2021']]

In [136]:
#check home values for nan
df2.isna().any()

Metropolitan statistical area    False
id                               False
ZillowName                       False
Latitude                         False
Longitude                        False
Home2000                          True
Home2008                         False
Home2017                         False
Home2018                         False
Home2019                         False
Home2020                         False
Home2021                         False
dtype: bool

In [137]:
#find home nans
df2.loc[df2['Home2000'].isna() == True]

,Metropolitan statistical area,id,ZillowName,Latitude,Longitude,Home2000,Home2008,Home2017,Home2018,Home2019,Home2020,Home2021
40,"Indianapolis-Carmel-Anderson, IN MSA",40,"Indianapolis, IN",39.74867,-86.212382,NaN,174055.6,176673.54,185627.03,194484.26,205721.63,218530.5


In [138]:
#replace nan with median for that year (since just 1 data point, this is the easiest)
df2['Home2000'].fillna(df2['Home2000'].median(),inplace=True)

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [140]:
#same as above figure but for home values

fig = go.Figure()

col_name = df2.columns
n_col = len(df2.columns)
date_list =[]
col_values = 5
n_range = 7

for i in range(n_range):
    df = df2[['Latitude','Longitude', 'ZillowName',col_name[col_values]]]
    lat = df['Latitude']
    lon = df['Longitude']

    df['text'] = df.apply(lambda x: x[2] + '<br> Home Value: $' + str(round(x[3],2)), axis=1)


    fig.add_trace(
    go.Scattergeo(
        name='',
        lon=lon,
        lat=lat,
        visible=False,
        hovertemplate=df['text'],
        showlegend=False,
        mode='markers',
        marker=dict(
            size=15,
            opacity=1,
            symbol='circle',
            reversescale = True,
            autocolorscale = False,
            colorscale = 'Hot',
            cmin = metro_data_long['Home'].min(),
            color = df[col_name[col_values]],
            cmax = 1000000,#metro_data_long['Home'].max(), #set this as a specific number to make higher values more obvious
            #cmid=300000.0,
            colorbar_title = "Home Values",
            #colorbar_dtick=.5,
            ),
        )
    )
    
    col_values += 1
    
steps =[]
years = metro_data_long['year'].sort_values().unique()

for i in range(len(fig.data)):
    value_avg = metro_data_long['Home'].loc[metro_data_long['year'] == years[i]].mean()
    step = dict(
        method='restyle',
        args = [
            {'visible': [False] * len(fig.data)}, #set all data to false visibility
            {'title': "<b>Home Values by Metro</b> <br><i>{0} Average: ${1}</i>".format(years[i], round(value_avg,2))}, #update title to include average for that year
        ],
        label = 'Year {}'.format(years[i]),
    )
    
    step["args"][0]["visible"][i] = True
    steps.append(step)
    

sliders = [dict(
    active=0,
    currentvalue = {'prefix':'Date: '},
    pad = {'t':1},
    steps = steps,
)]


fig.data[0].visible = True
min_year = metro_data_long['year'].min()
values_avg = metro_data_long['Home'].loc[metro_data_long['year'] == min_year].mean()

fig.update_layout(
    title = "<b>Home Values by Metro</b> <br><i>{0} Average: ${1}</i>".format(min_year, round(values_avg,2)),
    sliders=sliders,
    geo = dict(
        scope = 'usa',
        projection_type = 'albers usa',
        showland = True,
    )
)

fig.show()
#offline.plot(fig, filename='testhomevalues.html')

/Users/francescaortegren/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [98]:
underwaterdf = pd.read_json('underwater.json')

In [99]:
underwaterdf.head()

,State,County,percentage
0,Alabama,Russell,0.448457
1,Alabama,Lee,0.286311
2,Alabama,Etowah,0.253189
3,Alabama,Chambers,0.217227
4,Alabama,Calhoun,0.180075


In [106]:
underwaterdf['State'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [101]:
countycrosswalk = pd.read_csv('cbsatocountycrosswalk.csv')

In [103]:
countycrosswalk.dropna(subset=['cbsa'], inplace=True)

In [104]:
countycrosswalk.head()

,countyname,state,ssacounty,fipscounty,msa,l,msaname,cbsa,cbsaname,cbsaold,...,ssast,fipst,y2005,y2011,y2012,y2013,y2014,y2015,y2016,y2017
0,AUTAUGA,AL,1000,1001,5240.0,NaN,"MONTGOMERY, AL",33860.0,"Montgomery, AL",33860.0,...,1,1,2005.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0
3,BIBB,AL,1030,1007,1.0,NaN,ALABAMA,13820.0,"Birmingham-Hoover, AL",13820.0,...,1,1,2005.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0
4,BLOUNT,AL,1040,1009,1000.0,NaN,"BIRMINGHAM, AL",13820.0,"Birmingham-Hoover, AL",13820.0,...,1,1,2005.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0
7,CALHOUN,AL,1070,1015,450.0,NaN,"ANNISTON,AL",11500.0,"Anniston-Oxford, AL",11500.0,...,1,1,2005.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0
10,CHILTON,AL,1100,1021,1.0,NaN,ALABAMA,13820.0,"Birmingham-Hoover, AL",13820.0,...,1,1,2005.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0


In [107]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [108]:
underwaterdf['state'] = underwaterdf['State'].map(lambda x: us_state_to_abbrev[x])

In [109]:
underwaterdf.head()

,State,County,percentage,state
0,Alabama,Russell,0.448457,AL
1,Alabama,Lee,0.286311,AL
2,Alabama,Etowah,0.253189,AL
3,Alabama,Chambers,0.217227,AL
4,Alabama,Calhoun,0.180075,AL


In [110]:
countycrosswalk.rename(columns={'countyname':'County'}, inplace = True)

In [112]:
countycrosswalk['County'] = countycrosswalk['County'].str.lower()

In [113]:
underwaterdf['County'] = underwaterdf['County'].str.lower()

In [114]:
countycrosswalk.head()

,County,state,ssacounty,fipscounty,msa,l,msaname,cbsa,cbsaname,cbsaold,...,ssast,fipst,y2005,y2011,y2012,y2013,y2014,y2015,y2016,y2017
0,autauga,AL,1000,1001,5240.0,NaN,"MONTGOMERY, AL",33860.0,"Montgomery, AL",33860.0,...,1,1,2005.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0
3,bibb,AL,1030,1007,1.0,NaN,ALABAMA,13820.0,"Birmingham-Hoover, AL",13820.0,...,1,1,2005.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0
4,blount,AL,1040,1009,1000.0,NaN,"BIRMINGHAM, AL",13820.0,"Birmingham-Hoover, AL",13820.0,...,1,1,2005.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0
7,calhoun,AL,1070,1015,450.0,NaN,"ANNISTON,AL",11500.0,"Anniston-Oxford, AL",11500.0,...,1,1,2005.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0
10,chilton,AL,1100,1021,1.0,NaN,ALABAMA,13820.0,"Birmingham-Hoover, AL",13820.0,...,1,1,2005.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0


In [115]:
underwater = underwaterdf.merge(countycrosswalk[['County','state','msaname','cbsaname']], on=['County','state'], how='left')

In [117]:
underwater.groupby('cbsaname', as_index=False).mean()

,cbsaname,percentage
0,"Abilene, TX",0.039741
1,"Akron, OH",0.110980
2,"Albany, GA",0.264066
3,"Albany-Schenectady-Troy, NY",0.123337
4,"Albuquerque, NM",0.053284
...,...,...
369,"Yakima, WA",0.023803
370,"York-Hanover, PA",0.057253
371,"Youngstown-Warren-Boardman, OH-PA",0.145424
372,"Yuba City, CA",0.043707
